# Import

In [18]:
import numpy as np
import pandas as pd
import hvplot.pandas
import panel as pn
import param
import datetime as dt
import holoviews as hv
import requests
import dateutil.parser
import inspect
from datashader.colors import Sets1to3
from holoviews.operation.datashader import datashade
import datashader as ds
import intake
import os
#pn.extension('vtk')


# Lecture

In [20]:
# http://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+
from zipfile import ZipFile
import urllib.request

filename ='http://archive.ics.uci.edu/ml/machine-learning-databases/00357/occupancy_data.zip'
to_filename = "../data/occupancy_data.zip"

if not os.path.isfile(to_filename):
    print('Download')
    urllib.request.urlretrieve(filename, to_filename)

zip_file = ZipFile(to_filename)
dfs = {
    text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv') or text_file.filename.endswith('.txt')
}


In [21]:
df = dfs['datatest2.txt']
df.date = pd.to_datetime(df.date)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9752 entries, 1 to 9752
Data columns (total 7 columns):
date             9752 non-null datetime64[ns]
Temperature      9752 non-null float64
Humidity         9752 non-null float64
Light            9752 non-null float64
CO2              9752 non-null float64
HumidityRatio    9752 non-null float64
Occupancy        9752 non-null int64
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 609.5 KB


# Création class graph

In [3]:
def get_args(type_method):
    import re
    main_args = [arg for arg in inspect.getfullargspec(type_method).args if arg != 'self']
    all_args = {}

    regex_func = r"hvplot\.\w+\([\w+=, ]+\)"
    test_str =  inspect.getdoc(type_method)
    func_search = re.search(regex_func, test_str, re.IGNORECASE)
    if func_search:
        function = func_search.group(0)
        
        regex = r"([a-zA-z]+)=[a-zA-z]+"
        matches = re.finditer(regex, function, re.MULTILINE)
        
        for matchNum, match in enumerate(matches, start=1):
            key, val = match.group().split('=')
            all_args[key] = val
    return main_args, [arg for arg in all_args.keys() if arg not in main_args]

In [4]:
## Map
map_type_to_widget = {
    'C':'param.ObjectSelector(default="yellow", objects=["red","yellow","green"])',
    'c':'param.ObjectSelector(default="yellow", objects=["red","yellow","green"])',
    'text':'param.ObjectSelector(default="yellow", objects=["red","yellow","green"])',
    'bands':'param.ObjectSelector(default="yellow", objects=["red","yellow","green"])',
    'x':'param.ObjectSelector(default="yellow", objects=["red","yellow","green"])',
    'y':'param.ListSelector(default=["yellow"], objects=["red","yellow","green"])',
    'y2':'param.ObjectSelector(default="yellow", objects=["red","yellow","green"])',
    'by':'param.ObjectSelector(default="yellow", objects=["red","yellow","green"])',
    'z':'param.ObjectSelector(default="yellow", objects=["red","yellow","green"])',
    'colorbar':'param.ObjectSelector(default="yellow" ,objects=["red","yellow","green"])',
    'stacked':'param.Boolean(default=False)',
    'columns':'param.ObjectSelector(default="yellow", objects=["red","yellow","green"])',
    'where':'param.ObjectSelector(default="yellow", objects=["red","yellow","green"])'
}

In [16]:
hv_class = df.hvplot
members = []
all_args_types = []
all_types = []
all_opts_types = []
common_opts = []
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

def arg_to_widget_name(arg):
    try:
        return map_type_to_widget[arg]
    except Exception as e:
        return 'param.ObjectSelector(default="yellow",objects=["red","yellow","green"])'
    
for memb in inspect.getmembers(hv_class, predicate=inspect.ismethod):
    if '_' not in memb[0]:
        members.append(memb)
        
for graph_type in members:
    name = graph_type[0]
    my_class = name.capitalize()+'Panel'
    args, opts = get_args(eval('hv_class.'+graph_type[0]))
    common_opts = intersection(common_opts,opts)
    print('class '+my_class+'(param.Parameterized):\n')
    for arg in args:
        str_len = "        {:<"+str(len(str(max(args, key=len)))+1)+"}= "+arg_to_widget_name(arg)
        print(str_len.format(arg))
    print('')
    print('        #######################')
    print('        # Optional parameters #')
    print('        #######################')
    print('')
    for opt in opts:
        str_len = "        # {:<"+str(len(str(max(opts, key=len)))+1)+"}= "+arg_to_widget_name(opt)
        print(str_len.format(opt))
                            
    print('')
    print('        def __init__(self, dataframe=None, objects=None, defaults=None, **params):')
    print('            super('+my_class+', self).__init__(**params)\n')
    print('            if dataframe is not None:')
    print('                self.dataframe = dataframe')
    print('            else:')
    print('                self.dataframe = df')
    print('            for k, v in objects.items():')
    print('                try:')
    print('                    self.param[k].objects = v')
    print('                except:')
    print('                    pass')
    print('            for k, v in defaults.items():')
    print('                try:')
    print('                    setattr(self, k, v)')
    print('                except:')
    print('                    pass')

    
    print('            self.set_options()\n')
            
    print('        def set_options(self):')
    print('            self.options = {')
    for arg in args:
        str_len = "                {:<"+str(len(str(max(args, key=len)))+1)+"} : self."+arg+","
        print(str_len.format("'"+arg+"'"))
        
    for opt in opts:
        str_len = "                # {:<"+str(len(str(max(opts, key=len)))+1)+"} : self."+opt+","
        print(str_len.format("'"+opt+"'"))
    print('            }\n')

    print('        def view(self):')
    print('            self.set_options()')
    print('            return self.dataframe.hvplot.'+name+'(**self.options)\n')
    print(name+' = '+my_class+'(dataframe=df, objects=objects, defaults=defaults)')
    print('pn.Row('+name+', '+name+'.view)\n')


class AreaPanel(param.Parameterized):

        x       = param.ObjectSelector(default="yellow", objects=["red","yellow","green"])
        y       = param.ListSelector(default=["yellow"], objects=["red","yellow","green"])
        y2      = param.ObjectSelector(default="yellow", objects=["red","yellow","green"])
        stacked = param.Boolean(default=False)

        #######################
        # Optional parameters #
        #######################

        # alpha                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # color                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # fill_alpha              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # fill_color              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # hover_alpha             = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
   

[]

In [6]:
objects = {
    'x':list(df.columns), 
    'y':list(df.columns),
}
defaults = {
    'x':df.columns[0], 
    'y':[list(df.columns)[0]],
}
class LinePanel(param.Parameterized):

        x = param.ObjectSelector(default="yellow", objects=["red","yellow","green"])
        y = param.ListSelector(default=["yellow"], objects=["red","yellow","green"])

        #######################
        # Optional parameters #
        #######################

        # alpha                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # color                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # hover_alpha             = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # hover_color             = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # hover_line_alpha        = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # hover_line_color        = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_alpha              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_cap                = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_color              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_dash               = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_join               = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_width              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # muted_alpha             = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # muted_color             = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # muted_line_alpha        = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # muted_line_color        = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # nonselection_alpha      = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # nonselection_color      = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # nonselection_line_alpha = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # nonselection_line_color = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # selection_alpha         = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # selection_color         = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # selection_line_alpha    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # selection_line_color    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # width                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # height                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # shared_axes             = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # grid                    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # legend                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # rot                     = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # xlim                    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # ylim                    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # xticks                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # yticks                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # colorbar                = param.ObjectSelector(default="yellow" ,objects=["red","yellow","green"])
        # invert                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # title                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # logx                    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # logy                    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # loglog                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # xaxis                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # yaxis                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # xformatter              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # yformatter              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # xlabel                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # ylabel                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # padding                 = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # datashade               = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # rasterize               = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # x_sampling              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # y_sampling              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # aggregator              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        def __init__(self, dataframe=None, objects=None, defaults=None, **params):
            

            if dataframe is not None:
                self.dataframe = dataframe
            else:
                self.dataframe = df
            for k, v in objects.items():
                self.param[k].objects = v
            for k, v in defaults.items():
                setattr(self, k, v)
            self.set_options()
            super(LinePanel, self).__init__(**params)
        def set_options(self):
            self.options = {
                'x' : self.x,
                'y' : self.y,
                # 'alpha'                  : self.alpha,
                # 'color'                  : self.color,
                # 'hover_alpha'            : self.hover_alpha,
                # 'hover_color'            : self.hover_color,
                # 'hover_line_alpha'       : self.hover_line_alpha,
                # 'hover_line_color'       : self.hover_line_color,
                # 'line_alpha'             : self.line_alpha,
                # 'line_cap'               : self.line_cap,
                # 'line_color'             : self.line_color,
                # 'line_dash'              : self.line_dash,
                # 'line_join'              : self.line_join,
                # 'line_width'             : self.line_width,
                # 'muted_alpha'            : self.muted_alpha,
                # 'muted_color'            : self.muted_color,
                # 'muted_line_alpha'       : self.muted_line_alpha,

                # 'muted_line_color'       : self.muted_line_color,
                # 'nonselection_alpha'     : self.nonselection_alpha,
                # 'nonselection_color'     : self.nonselection_color,
                # 'nonselection_line_alpha' : self.nonselection_line_alpha,
                # 'nonselection_line_color' : self.nonselection_line_color,
                # 'selection_alpha'        : self.selection_alpha,
                # 'selection_color'        : self.selection_color,
                # 'selection_line_alpha'   : self.selection_line_alpha,
                # 'selection_line_color'   : self.selection_line_color,
                # 'width'                  : self.width,
                # 'height'                 : self.height,
                # 'shared_axes'            : self.shared_axes,
                # 'grid'                   : self.grid,
                # 'legend'                 : self.legend,
                # 'rot'                    : self.rot,
                # 'xlim'                   : self.xlim,
                # 'ylim'                   : self.ylim,
                # 'xticks'                 : self.xticks,
                # 'yticks'                 : self.yticks,
                # 'colorbar'               : self.colorbar,
                # 'invert'                 : self.invert,
                # 'title'                  : self.title,
                # 'logx'                   : self.logx,
                # 'logy'                   : self.logy,
                # 'loglog'                 : self.loglog,
                # 'xaxis'                  : self.xaxis,
                # 'yaxis'                  : self.yaxis,
                # 'xformatter'             : self.xformatter,
                # 'yformatter'             : self.yformatter,
                # 'xlabel'                 : self.xlabel,
                # 'ylabel'                 : self.ylabel,
                # 'padding'                : self.padding,
                # 'datashade'              : self.datashade,
                # 'rasterize'              : self.rasterize,
                # 'x_sampling'             : self.x_sampling,
                # 'y_sampling'             : self.y_sampling,
                # 'aggregator'             : self.aggregator,
            }

        def view(self):
            self.set_options()
            return self.dataframe.hvplot.line(**self.options)

line = LinePanel(dataframe=df, objects=objects, defaults=defaults)
pn.Row(line, line.view)


Row
    [0] Column(margin=5, name='Param01318', width=300)
        [0] StaticText(value='<b>LinePanel</b>')
        [1] Select(name='X', options=OrderedDict([('Date', ...]), value='Date')
        [2] MultiSelect(name='Y', options=OrderedDict([('Date', ...]), value=['Date'])
    [1] ParamMethod(method)

In [7]:
class LinePanelOptions(param.Parameterized):

        #######################
        # Optional parameters #
        #######################

        alpha                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        color                   = param.Color(default='#FFFFFF')
        # hover_alpha             = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # hover_color             = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # hover_line_alpha        = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # hover_line_color        = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_alpha              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_cap                = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_color              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_dash               = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_join               = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # line_width              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # muted_alpha             = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # muted_color             = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # muted_line_alpha        = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # muted_line_color        = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # nonselection_alpha      = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # nonselection_color      = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # nonselection_line_alpha = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # nonselection_line_color = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # selection_alpha         = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # selection_color         = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # selection_line_alpha    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # selection_line_color    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # width                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # height                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # shared_axes             = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # grid                    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # legend                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # rot                     = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # xlim                    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # ylim                    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # xticks                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # yticks                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # colorbar                = param.ObjectSelector(default="yellow" ,objects=["red","yellow","green"])
        # invert                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # title                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # logx                    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # logy                    = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # loglog                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # xaxis                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # yaxis                   = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # xformatter              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # yformatter              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # xlabel                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # ylabel                  = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # padding                 = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # datashade               = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # rasterize               = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # x_sampling              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # y_sampling              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])
        # aggregator              = param.ObjectSelector(default="yellow",objects=["red","yellow","green"])        
        
        
        def __init__(self, **params): #, dataframe=None, objects=None, defaults=None,
            super(LinePanelOptions, self).__init__(**params)
            self.set_options()

        def set_options(self):
            self.options = {
                 'alpha'                  : self.alpha,
                 'color'                  : self.color,
           
                # 'alpha'                  : self.alpha,
                # 'color'                  : self.color,
                # 'hover_alpha'            : self.hover_alpha,
                # 'hover_color'            : self.hover_color,
                # 'hover_line_alpha'       : self.hover_line_alpha,
                # 'hover_line_color'       : self.hover_line_color,
                # 'line_alpha'             : self.line_alpha,
                # 'line_cap'               : self.line_cap,
                # 'line_color'             : self.line_color,
                # 'line_dash'              : self.line_dash,
                # 'line_join'              : self.line_join,
                # 'line_width'             : self.line_width,
                # 'muted_alpha'            : self.muted_alpha,
                # 'muted_color'            : self.muted_color,
                # 'muted_line_alpha'       : self.muted_line_alpha,

                # 'muted_line_color'       : self.muted_line_color,
                # 'nonselection_alpha'     : self.nonselection_alpha,
                # 'nonselection_color'     : self.nonselection_color,
                # 'nonselection_line_alpha' : self.nonselection_line_alpha,
                # 'nonselection_line_color' : self.nonselection_line_color,
                # 'selection_alpha'        : self.selection_alpha,
                # 'selection_color'        : self.selection_color,
                # 'selection_line_alpha'   : self.selection_line_alpha,
                # 'selection_line_color'   : self.selection_line_color,
                # 'width'                  : self.width,
                # 'height'                 : self.height,
                # 'shared_axes'            : self.shared_axes,
                # 'grid'                   : self.grid,
                # 'legend'                 : self.legend,
                # 'rot'                    : self.rot,
                # 'xlim'                   : self.xlim,
                # 'ylim'                   : self.ylim,
                # 'xticks'                 : self.xticks,
                # 'yticks'                 : self.yticks,
                # 'colorbar'               : self.colorbar,
                # 'invert'                 : self.invert,
                # 'title'                  : self.title,
                # 'logx'                   : self.logx,
                # 'logy'                   : self.logy,
                # 'loglog'                 : self.loglog,
                # 'xaxis'                  : self.xaxis,
                # 'yaxis'                  : self.yaxis,
                # 'xformatter'             : self.xformatter,
                # 'yformatter'             : self.yformatter,
                # 'xlabel'                 : self.xlabel,
                # 'ylabel'                 : self.ylabel,
                # 'padding'                : self.padding,
                # 'datashade'              : self.datashade,
                # 'rasterize'              : self.rasterize,
                # 'x_sampling'             : self.x_sampling,
                # 'y_sampling'             : self.y_sampling,
                # 'aggregator'             : self.aggregator,
            }


            
class LinePanel(param.Parameterized):
    
    x = param.ObjectSelector(default="yellow", objects=["red","yellow","green"])
    y = param.ListSelector(default=["yellow"], objects=["red","yellow","green"])
    
    options = param.Parameter(precedence=3)

    def __init__(self, dataframe=None, objects=None, defaults=None, **params):
        if 'options' not in params:
            params['options'] = LinePanelOptions(name='Options')

                
        if dataframe is not None:
            self.dataframe = dataframe
        else:
            self.dataframe = df
        for k, v in objects.items():
            self.param[k].objects = v
        for k, v in defaults.items():
            setattr(self, k, v)

        self.set_options()
        super(LinePanel, self).__init__(**params)
    def set_options(self):
        self.options = {
            'x' : self.x,
            'y' : self.y
        }
        
        
    def view(self):
        return df.hvplot(x=self.x, y=self.y)
    
line = LinePanel(dataframe=df, objects=objects, defaults=defaults)

subpanel = pn.Column()
pn.Row(
    pn.Column(pn.Param(line.param, expand_layout=subpanel), subpanel), line.view
)


Row
    [0] Column
        [0] Param(LinePanel, expand_layout=Column())
        [1] Column()
    [1] ParamMethod(method)